# Создание ROS узлов с использованием rospy

Самое время познакомиться с возможностями написания узлов на языке Python2. В качестве основы можно посмотреть офф страницу о [написании узлов](http://wiki.ros.org/rospy_tutorials/Tutorials/WritingPublisherSubscriber).

Для начала, импортируем основной модуль `rospy` и модуль сообщения типа `std_msgs/String`. 

In [ ]:
import rospy
from std_msgs.msg import String

После этого необходимо зарегистрировать узел в системе ROS, а также зарегистрировать топик на публикацию с указанием имени, типа сообщения для топика и размера очереди. Очередь нужна для сохранения сообщений, если узел публикует сообщения часто, при этом низкоуровневая передача сообщений работает медленнее или с задержками. При переполнении очереди отправляются наиболее актуальные данные.

In [ ]:
rospy.init_node('talker')
pub = rospy.Publisher('my_chat_topic', String, queue_size=10)

После остается только создать объект `Rate`, который используется для выдерживания частоты выполнения кода. В конструктор передается значение частоты в Гц.

In [ ]:
rate = rospy.Rate(1) # 1 Hz

На этом подготовка и создание необходимых объектов для простейшего узла готовы и пора перейти к основной логике программы.

В API ROS есть функция, которая сообщает о том, что система ROS завершила работу, именно ей и воспользуемся в качестве условия выхода `rospy.is_shutdown()`. Далее определим функцию с основной логикой узла для дальнейшего запуска.

In [ ]:
def talker():
    while not rospy.is_shutdown():
        # Сформируем сообщение, которое включает время
        hello_str = "hi =) %s" % rospy.get_time()
        # Вывод в терминал информации (содержание сообщения)
        rospy.loginfo(hello_str)
        # Публикация сообщения в топик
        pub.publish(hello_str)
        # Сон в соответствии с выдерживаемой частотой
        rate.sleep()

В случае с типом данных `std_msgs/String` можно просто передавать в функцию `publish()` данные сообщения. В случае с более комплексными данными рекомендуется сначала создавать объект сообщения, заполнять его и после публиковать.

```python
msg = String()
msg.data = hello_str

pub.publish(msg)
```

или

```python
msg = String(data.hello_str)

pub.publish(msg)
```

После этого можем запустить функцию узла (в ней находится вся логика). При этом заворачиваем в конструкцию `try-catch`, чтобы обработать прерывание (нажатием Стоп или Ctrl+C к терминале). 

In [ ]:
try:
    talker()
except (rospy.ROSInterruptException, KeyboardInterrupt):
    pass

Общий код узла:
```python
import rospy
from std_msgs.msg import String

rospy.init_node('talker')
pub = rospy.Publisher('chatter', String, queue_size=10)
rate = rospy.Rate(1) # 1 Hz

def talker():
    while not rospy.is_shutdown():
        # Сформируем сообщение, которое включает время
        hello_str = "hi =) %s" % rospy.get_time()
        # Вывод в терминал информации (содержание сообщения)
        rospy.loginfo(hello_str)
        # Публикация сообщения в топик
        pub.publish(hello_str)
        # Сон в соответствии с выдерживаемой частотой
        rate.sleep()

try:
    talker()
except (rospy.ROSInterruptException, KeyboardInterrupt):
    pass
```

> Задачка по самостоятельной интеграции скрипта внутри пакета.
- Внутри пакета создать папку `scripts` (Python файлы считаются скриптами), в ней создать файл talker.py и в нем разместить код узла.
- Далее дать права на выполение с помощью команды `chmod +x talker.py`
- Попробовать запустить в системе ROS созданный узел, для ранее созданного пакета команда будет следующей:  
`rosrun my_new_package talker.py`
- Прим.: так как мы не задавали флаг анонимности в функции `rospy.init_node('talker')` (по-умолчанию там стоит `anonymous=False`), явно присваивать имя узла не требуется, оно будет такое, как было задано в функции `rospy.init_node()`

## В результате

- Был создан узел, публикующий сообщения типа строки. Рассмотрено основное API пакета rospy.